In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import requests
from bs4 import BeautifulSoup

url1 = "https://www.cars24.com/buy-used-cars-hyderabad/"
url2='https://www.cars24.com/buy-used-car?sort=bestmatch&serveWarrantyCount=true&gaId=545137534.1693820298&storeCityId=4709&pinId=560001'
chrome_driver_path = "C:/Program Files/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

driver.get(url1)

scroll_count = 300

for i in range(scroll_count):

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(0.3)

a1 = []
b1 = []
d1 = []
c1=[]

data = driver.page_source
sdata = BeautifulSoup(data, 'html.parser')
a = sdata.find_all('h2', attrs={'class': '_2lmIw'})#name,year
b = sdata.find_all('ul', attrs={'class': '_1hOnS'})#engine and transmission
d = sdata.find_all('ul', attrs={'class': "_13yb6"})#kms, owner, petrol/diesel
c = sdata.find_all('div', attrs={'class': "_18ToE"})#price

for i in range(len(a)):
    a1.append(a[i].text)
    b1.append(b[i].text)
    d1.append(d[i].text)
    c1.append(c[i].text)

driver.quit()




In [ ]:
#url2
chrome_driver_path = "C:/Program Files/chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
driver.get(url2)
for i in range(scroll_count):

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(0.3)
data = driver.page_source
sdata = BeautifulSoup(data, 'html.parser')
a = sdata.find_all('h2', attrs={'class': '_2lmIw'})#name,year
b = sdata.find_all('ul', attrs={'class': '_1hOnS'})#engine and transmission
d = sdata.find_all('ul', attrs={'class': "_13yb6"})#kms, owner, petrol/diesel
c = sdata.find_all('div', attrs={'class': "_18ToE"})#price

for i in range(len(a)):
    a1.append(a[i].text)
    b1.append(b[i].text)
    d1.append(d[i].text)
    c1.append(c[i].text)

driver.quit()

In [ ]:
len(a1)

1950

In [ ]:
#adding year of release
e1=[]#year will be stored
for i in range(len(a1)):
  e1.append(int(a1[i][:4]))
  a1[i]=a1[i][4:]

In [ ]:
# separating engine and transmission(b)
f1=[]#transmission will be stored
for i in range(len(a1)):
  if('Manual' in b1[i]):
    f1.append('Manual')
    b1[i]=b1[i].replace('Manual',"")
  else:
    f1.append("Automatic")
    b1[i]=b1[i].replace('Automatic',"")
# engine type in b1

In [ ]:
# removing trash
d2=d1.copy()

#kms nth hand  fuel (d)
for i in range(len(a1)):
  d2[i]= d2[i][-5:]
  d1[i]=d1[i].replace(d2[i],"")


In [ ]:
g1=[]# for fuel type
for i in range(len(a1)):
  g1.append( d1[i][-6:])
  d1[i]= d1[i].replace(g1[i],"")

In [ ]:
h1=[] # ownership
for i in range(len(a1)):
  h1.append(d1[i][-9:])
  d1[i]= d1[i].replace(h1[i],"")

In [ ]:
for i in range(len(a1)): #ownership error adjustment
  if(' km1st Ow' in h1[i]):
    h1[i]=h1[i].replace(' km1st Ow','1st Owner')

In [ ]:
for i in range(len(a1)):
  if(' km' in d1[i]):
    d1[i]= d1[i].replace(' km',"")
    d1[i]=d1[i].replace(',','')
    d1[i]=int(d1[i])
#d1 contains kms travelled

In [ ]:
#adjusting actual price
for i in range(len(a1)):
  if(len(c1[i])==18):
    c1[i]= c1[i].replace(c1[i][-9:],"")
    c1[i]= c1[i].replace('₹',"")
    c1[i]= c1[i].replace(',',"")
  elif len(c1[i])>=19:
    c1[i]= c1[i].replace(c1[i][-10:],"")
    c1[i]= c1[i].replace('₹',"")
    c1[i]= c1[i].replace(',',"")
  else:
    c1[i]= c1[i].replace('₹',"")
    c1[i]= c1[i].replace(',',"")

In [ ]:
df= pd.DataFrame({'name':a1,'year':e1,'engine':b1,'transmission':f1,'fuel':g1,'ownership':h1,'distance_travelled':d1,'Price':c1})

In [ ]:
df

,name,year,engine,transmission,fuel,ownership,distance_travelled,Price
0,Nissan MAGNITE,2021,XE,Manual,Petrol,1st Owner,18745,597000
1,Hyundai Elite i20,2015,MAGNA 1.2,Manual,Petrol,1st Owner,58583,500000
2,Hyundai Creta,2018,SX PLUS 1.6 PETROL,Manual,Petrol,1st Owner,81644,999000
3,Volkswagen Polo,2021,HIGHLINE PLUS 1.0L TSI AT,Automatic,Petrol,1st Owner,55053,944000
4,Hyundai Verna,2020,SX 1.5 VTVT,Manual,Petrol,1st Owner,15471,1033000
...,...,...,...,...,...,...,...,...
1945,Maruti S PRESSO,2020,VXI PLUS AMT,Automatic,Petrol,1st Owner,18691,546000
1946,Toyota Glanza,2022,V,Manual,Petrol,1st Owner,12786,940000
1947,Hyundai i20 Active,2017,1.2 SX,Manual,Petrol,1st Owner,43031,714000
1948,Maruti Wagon R 1.0,2010,VXI,Manual,Petrol,1st Owner,131140,250000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                1950 non-null   object
 1   year                1950 non-null   int64 
 2   engine              1950 non-null   object
 3   transmission        1950 non-null   object
 4   fuel                1950 non-null   object
 5   ownership           1950 non-null   object
 6   distance_travelled  1950 non-null   object
 7   Price               1950 non-null   object
dtypes: int64(1), object(7)
memory usage: 122.0+ KB


In [ ]:
df.to_csv('car_dataset_updated.csv')